In [2]:
device = "cuda" if torch.cuda.is_available else "cpu"
torch.set_default_device(device)
torch.manual_seed(42)

device

'cuda'

In [3]:
import random
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, ConcatDataset

generator = torch.Generator(device=device)
# Define augmentation transforms
rotation_degrees = 20
translation_percent = 0.1
validation_ratio = 0.2

transform_list = [
    transforms.RandomRotation(degrees=(-rotation_degrees, rotation_degrees)),
    transforms.RandomAffine(degrees=0, translate=(translation_percent, translation_percent)),
    transforms.RandomVerticalFlip(p=0.5),
    
]

# Path and configurations
data_dir = "./TRAIN/"
batch_size = 400

# Original dataset without augmentation
original_dataset = ImageFolder(root=data_dir, transform=transforms.ToTensor())

augmented = []
for index, (data, label) in enumerate(original_dataset):
    images = []
    for transform in transform_list:
        if random.random() > 0.15:
            transformed_data = transform(data)
            images.append(transformed_data)
            if len(images) >= 2 and random.random() > 0.50:
                transformed_data = transform(images[-2])
                images.append(transformed_data)
    else:
        for image in images:  augmented.append((image, label))
        
        
        

class AugmentedCustomDataset(Dataset):
    def __init__(self, augmented_data):
        self.augmented_data = augmented_data

    def __len__(self):
        return len(self.augmented_data)

    def __getitem__(self, idx):
        images, label = self.augmented_data[idx]
        tensor_images = [torch.Tensor(image) for image in images]
        combined_images = torch.stack(tensor_images)
        
        return combined_images, label

custom_augmented_dataset = AugmentedCustomDataset(augmented)
combined_dataset = torch.utils.data.ConcatDataset([original_dataset, custom_augmented_dataset])

val_size = int(len(combined_dataset) * validation_ratio)
train_size = int(len(combined_dataset) - val_size)


In [4]:
generator = torch.Generator(device=device)
train_dataset, val_dataset = torch.utils.data.random_split(combined_dataset, [train_size, val_size], generator=generator)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=400, shuffle=True,generator=torch.Generator(device='cuda'))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=400, shuffle=False,generator=torch.Generator(device='cuda'))

In [5]:


data_dir = './TUNE/'
data_transforms = transforms.Compose([
    transforms.ToTensor()
])
custom_dataset = ImageFolder(root=data_dir, transform=data_transforms)

batch_size = 400 
val_size = int(len(custom_dataset) * 0.1)
train_size = int(len(custom_dataset) - val_size)


generator = torch.Generator(device=device)
train_dataset_ft, val_dataset_ft = torch.utils.data.random_split(custom_dataset, [train_size, val_size],generator=generator)
train_loader_ft = torch.utils.data.DataLoader(train_dataset_ft, batch_size=1, shuffle=True,generator=torch.Generator(device='cuda'))
val_loader_ft = torch.utils.data.DataLoader(val_dataset_ft, batch_size=1, shuffle=False,generator=torch.Generator(device='cuda'))

In [32]:
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=in_channels)
        self.bn1 = nn.BatchNorm2d(in_channels)

        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):

        x = self.depthwise(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.pointwise(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x

class CustomDecoder(nn.Module):
    def __init__(self, num_classes=8):
        super(CustomDecoder, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)
        self.dconv1 = DepthwiseSeparableConv(32, 64, kernel_size=3, stride=1, padding=1)
        self.dconv2 = DepthwiseSeparableConv(64, 128, kernel_size=3, stride=2, padding=1)
        self.dconv3 = DepthwiseSeparableConv(128, 128, kernel_size=3, stride=1, padding=1)
        self.dconv4 = DepthwiseSeparableConv(128, 256, kernel_size=3, stride=2, padding=1)
        self.dconv5 = DepthwiseSeparableConv(256, 256, kernel_size=3, stride=1, padding=1)
        
        self.dconv6 = DepthwiseSeparableConv(256, 512, kernel_size=3, stride=2, padding=1)
        self.fivex_dconv = nn.ModuleList([DepthwiseSeparableConv(512, 512, kernel_size=3, stride=1, padding=1) for _ in range(5)])
        
        self.dconv7 = DepthwiseSeparableConv(512, 1024, kernel_size=3, stride=2, padding=1)
        self.dconv8 = DepthwiseSeparableConv(1024, 1024, kernel_size=3, stride=1, padding=1)
        
        self.dropout_dense = nn.Dropout(p=0.3)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.dconv1(x)
        x = self.dconv2(x)
        x = self.dconv3(x)
        x = self.dconv4(x)
        x = self.dconv5(x)
        x = self.dconv6(x)
        for dconv_layer in self.fivex_dconv: x = dconv_layer(x)   
        x = self.dconv7(x)
        x = self.dconv8(x)
        x = self.global_avg_pool(x)
        x = self.dropout_dense(x)
        x = torch.flatten(x, 1) 
        x = self.fc(x)
        return x




In [33]:
from torch import optim
alex_net = CustomDecoder().to(device).float()
optimizer = optim.Adam(alex_net.parameters(), lr=0.001) 
criterion = nn.CrossEntropyLoss()

In [34]:
num_epochs = 24

for epoch in range(num_epochs):
    alex_net.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = alex_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    alex_net.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = alex_net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val

    epoch_loss = running_loss / len(train_loader)
    epoch_val_loss = val_loss / len(val_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {epoch_loss:.4f} - Val Loss: {epoch_val_loss:.4f} - Train Acc: {train_accuracy:.2f}% - Val Acc: {val_accuracy:.2f}%")

Epoch [1/24] - Loss: 1.9964 - Val Loss: 2.0774 - Train Acc: 18.88% - Val Acc: 20.39%


KeyboardInterrupt: 

In [ ]:
for epoch in range(16):
    alex_net.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    

    for inputs, labels in train_loader_ft:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = alex_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    alex_net.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader_ft:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = alex_net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val

    epoch_loss = running_loss / len(train_loader_ft)
    epoch_val_loss = val_loss / len(val_loader_ft)
    print(f"Epoch [{epoch + 1}/{16}] - Loss: {epoch_loss:.4f} - Val Loss: {epoch_val_loss:.4f} - Train Acc: {train_accuracy:.2f}% - Val Acc: {val_accuracy:.2f}%")